In [1]:
import torchvision
from torchvision.datasets.video_utils import VideoClips
from torchvision.datasets.utils import list_dir
from torchvision.datasets.folder import make_dataset
from torchvision.datasets.vision import VisionDataset

ImportError: No module named 'torchvision.datasets.video_utils'

In [4]:
from torchvision.datasets import video_utils

ImportError: cannot import name 'video_utils'

In [ ]:



class KineticsVideo(VisionDataset):
    def __init__(self, root, frames_per_clip, step_between_clips=1):
        super(KineticsVideo, self).__init__(root)
        extensions = ('avi',)

        classes = list(sorted(list_dir(root)))
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        self.samples = make_dataset(self.root, class_to_idx, extensions, is_valid_file=None)
        self.classes = classes
        video_list = [x[0] for x in self.samples]
        self.video_clips = VideoClips(video_list, frames_per_clip, step_between_clips)

    def __len__(self):
        return self.video_clips.num_clips()

    def __getitem__(self, idx):
        video, audio, info, video_idx = self.video_clips.get_clip(idx)
        label = self.samples[video_idx][1]

        return video, audio, label

In [ ]:
import os, io, shutil
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm

import numpy as np
from glob import glob
from tensorboardX import SummaryWriter
from dataloader.handhygiene import HandHygiene
#from dataloader.i3ddataset import I3DDataset
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, RandomAffine, #ExtractSkinColor,
    RandomRotation, ColorJitter, ToTensor)
from temporal_transforms import (
    MirrorPadding, LoopPadding, TemporalBeginCrop, 
    TemporalRandomCrop, TemporalCenterCrop, TemporalRandomChoice)
from openpose_transforms import CropTorso, MultiScaleTorsoRandomCrop

In [ ]:
model_name = 'i3d'
batch_size = 16
clip_len = 16
sample_duration = clip_len
num_classes = 1

#torch.manual_seed(100)
data_name = 'anesthesia'
dataset_path = os.path.join(os.getcwd(), 'data')

In [ ]:
sample_size = 224
#mean=[110.63666788, 103.16065604, 96.29023126]
#std=[38.7568578, 37.88248729, 40.02898126]
mean=[128, 128, 128]
std=[128, 128, 128]

scales=np.linspace(1, 1.75, num=1e3)
center=((1+1.75)/2)
openpose_transform = {
    'train':MultiScaleTorsoRandomCrop(scales, sample_size),
    'val':MultiScaleTorsoRandomCrop(np.linspace(center, center, num=1), sample_size, centercrop=True)
}
spatial_transform = {
    'train': Compose([Scale(sample_size),
                      CenterCrop(sample_size),
                      RandomHorizontalFlip(),
                      ColorJitter(brightness=0.1),
                      RandomAffine(5),
                      RandomRotation(2.5),
                      ToTensor(1), 
                      Normalize(mean, std)]),
    'val': Compose([Scale(sample_size), 
                    CenterCrop(sample_size), 
                    ToTensor(1), 
                    Normalize(mean, std)])}
temporal_transform = {
    'train': TemporalRandomChoice([
            TemporalBeginCrop(sample_duration),
            MirrorPadding(sample_duration)]),
    'val':TemporalBeginCrop(sample_duration)}

In [ ]:
dataset = {
    'train':HandHygiene(dataset_path, split='train', clip_len=clip_len, 
                        spatial_transform=spatial_transform['train'],
                        openpose_transform=openpose_transform['train'],
                        temporal_transform=temporal_transform['train'], cropped=False),
    'val':HandHygiene(dataset_path, split='val', clip_len=clip_len, 
                        openpose_transform=openpose_transform['val'],
                        spatial_transform=spatial_transform['val'],
                        temporal_transform=temporal_transform['val'], cropped=False),
    'test':HandHygiene(dataset_path, split='test', clip_len=clip_len, 
                        openpose_transform=openpose_transform['val'],
                        spatial_transform=spatial_transform['val'],
                        temporal_transform=temporal_transform['val'], cropped=False)}

dataloaders = {
    'train': DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=16),
    'val': DataLoader(dataset['val'], batch_size=batch_size, shuffle=False, num_workers=16)}

# Training

In [ ]:
from train import get_models
from train import train
from torchsummary import summary

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def show_dataset(item): # item: C x D x 244 x 244
    rgbs = item[0].transpose(0, 1)
    flows = item[1].transpose(0, 1)
    n = len(rgbs)
    shape = np.asarray(flows[0]).shape
    rgb = np.hstack((np.asarray(rgb).transpose(1, 2, 0)+1)/2 for rgb in rgbs)
    #gray = np.hstack((np.squeeze(np.asarray(gray))+1)/2 for gray in rgbs)
    tmp = np.zeros((shape[1], shape[2], 1))
    flow = np.hstack((np.dstack((np.asarray(flow).transpose(1, 2, 0), tmp))+1)/2 for flow in flows)

    img = np.vstack((rgb, flow))
    plt.figure(figsize=(50, 10))
    plt.imshow(img)
    #plt.imshow(gray)
    plt.axis('off')
    plt.show()
    print(item[2])

i=1
phase='val'
show_dataset(dataset[phase].__getitem__(i))
print(dataset[phase].__getpath__(i))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
i3d_rgb, i3d_flow = get_models(num_classes, True, 170, load_pt_weights=True) # unfreeze last mix 170, 152
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    i3d_rgb = nn.DataParallel(i3d_rgb)
    i3d_flow = nn.DataParallel(i3d_flow)
i3d_rgb.to(device)
i3d_flow.to(device)

criterion = F.binary_cross_entropy
optims={'rgb':None, 'flow':None}
schedulers = {'rgb':None, 'flow':None}
feature_extract=True

In [ ]:
def trainable_params(model, mode='rgb'):
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    optims[mode] = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9, weight_decay=1e-7)

trainable_params(i3d_rgb, 'rgb')
trainable_params(i3d_flow, 'flow')
    
schedulers['rgb'] = MultiStepLR(optims['rgb'], milestones=[10], gamma=0.1)
schedulers['flow'] = MultiStepLR(optims['flow'], milestones=[10], gamma=0.1)

In [ ]:
#summary(i3d_rgb, (3, 16, 224, 224))

In [ ]:
train((i3d_rgb, i3d_flow), dataloaders, optims, criterion, schedulers, device, num_epochs=200)

# 5-Fold Validation

In [ ]:
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
with open('data/list_simulation.txt', 'r') as f:
    data_all = f.readlines()
data_all = [os.path.join('./data/images/train/clean/', path.strip('\n')) for path in data_all]
data_train, data_test = train_test_split(data_all[16:], test_size=0.2, random_state=42)
for path in data_all[:16]:
    src = path+'*'
    dir_val = './data/images/test/clean'
    #!mv $src $dir_val

In [ ]:
data_train, data_test = train_test_split(data_notclean, test_size=0.2, random_state=42)

# 데이터 솎아내기

In [ ]:
for phase in ['test']:#['train', 'val', 'test']:
    data = sorted(glob('./data/images/{}/notclean/*'.format(phase)))
    data = [path.replace('{}/notclean'.format(phase), 'notclean') for path in data]
    data_all=[]
    for path in data:
        augs = sorted(glob(path+'*'))
        num=1
        if len(augs[::num]) > 30:
            num=2
        for aug in augs[::num]:
            data_all.append(aug)
    print(len(data_all))

In [ ]:
for path in data_all:
    !mv $path ./data/images/test/notclean/

# EDA

In [ ]:
from glob import glob
clean = {'train':[], 'val':[], 'test':[]}
notclean = {'train':[], 'val':[], 'test':[]}
clean_num = {'train':[], 'val':[], 'test':[]}
notclean_num = {'train':[], 'val':[], 'test':[]}

for phase in ['train', 'val', 'test']:
    paths = [path for path in dataset[phase].samples[0] if len(os.path.basename(path).split('_')) == 3]
    clean[phase] = [path for path in paths if '/clean/' in path]
    notclean[phase] = [path for path in paths if '/notclean/' in path]
for phase in ['train', 'val', 'test']:
    for f in clean[phase]: 
        num = len(glob(os.path.join(f, '*.jpg')))
        clean_num[phase].append(num)
    for f in notclean[phase]: 
        num = len(glob(os.path.join(f, '*.jpg')))
        notclean_num[phase].append(num)

print('     ', 'clips', 'images', 'min', 'max')
for key, value in clean_num.items():
    print(key, len(value), sum(value), min(value), max(value))
print('     ', 'clips', 'images', 'min', 'max')
for key, value in notclean_num.items():
    print(key, len(value), sum(value), min(value), max(value))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def draw_result(lst_iter, lst_loss, lst_acc, title):
    plt.plot(lst_iter, lst_loss, '-b', label='loss')
    plt.plot(lst_iter, lst_acc, '-r', label='accuracy')

    plt.xlabel("n iteration")
    plt.legend(loc='upper left')
    plt.title(title)

    # save image
    plt.savefig(title+".png")  # should before show method

    # show
    plt.show()


def test_draw():
    # iteration num
    lst_iter = range(100)

    # loss of iteration
    lst_loss = [0.01 * i + 0.01 * i ** 2 for i in xrange(100)]
    # lst_loss = np.random.randn(1, 100).reshape((100, ))

    # accuracy of iteration
    lst_acc = [0.01 * i - 0.01 * i ** 2 for i in xrange(100)]
    # lst_acc = np.random.randn(1, 100).reshape((100, ))
    draw_result(lst_iter, lst_loss, lst_acc, "sgd_method")


if __name__ == '__main__':
    test_draw()